# Human in the Loop  

Human in the Loop (HITL) means involving humans during an AI system’s decision-making.  
It ensures oversight, correction, and control over automated outputs.  
This approach balances automation with human judgment for safer results.  


Docuumentation Link: https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/add-human-in-the-loop/#add-interrupts-to-any-tool


In [1]:
!pip install langchain_core

In [1]:
from typing import Callable
from langchain_core.tools import BaseTool, tool as create_tool
from langchain_core.runnables import RunnableConfig

def add_human_in_the_loop(tool: Callable | BaseTool) -> BaseTool:
    """Wrap a tool to support human-in-the-loop review in Jupyter Notebook."""
    if not isinstance(tool, BaseTool):
        tool = create_tool(tool)

    @create_tool(
        tool.name,
        description=tool.description,
        args_schema=tool.args_schema
    )
    def call_tool_with_interrupt(config: RunnableConfig, **tool_input):
        print("\n--- HUMAN IN THE LOOP ---")
        print(f"Action: {tool.name}")
        print(f"Args: {tool_input}")
        user_choice = input("Type 'accept', 'edit', or 'response': ").strip()

        if user_choice == "accept":
            tool_response = tool.invoke(tool_input, config)

        elif user_choice == "edit":
            for key, val in tool_input.items():
                new_val = input(f"Enter new value for {key} (leave blank to keep '{val}'): ")
                if new_val.strip():
                    tool_input[key] = new_val
            tool_response = tool.invoke(tool_input, config)

        elif user_choice == "response":
            user_feedback = input("Enter your feedback: ")
            tool_response = user_feedback

        else:
            raise ValueError(f"Unsupported response type: {user_choice}")

        return tool_response

    return call_tool_with_interrupt


In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    api_key="<API_KEY>",
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)


In [3]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

checkpointer = InMemorySaver()

def book_hotel(hotel_name: str):
   """Book a hotel"""
   return f"Successfully booked a stay at {hotel_name}."

def check_weather(hotel_name: str):
   """Check the weather for a hotel"""
   return f"The weather at {hotel_name} is sunny."


agent = create_react_agent(
    model=llm,
    tools=[
        add_human_in_the_loop(book_hotel),
        add_human_in_the_loop(check_weather),
    ],
    checkpointer=checkpointer,
)


In [5]:

config = {"configurable": {"thread_id": "1"}}

# Run the agent
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "book a stay at McKittrick hotel"}]},
    config
):
    print(chunk)
    print("\n")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_hotel', 'arguments': '{"hotel_name": "McKittrick hotel"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ac9dbf77-d6de-4d4e-981d-eb8886a49bb5-0', tool_calls=[{'name': 'book_hotel', 'args': {'hotel_name': 'McKittrick hotel'}, 'id': 'cf91d58a-c7ed-4d26-8906-db4c9bc83289', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 75, 'total_tokens': 234, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 55}})]}}



--- HUMAN IN THE LOOP ---
Action: book_hotel
Args: {'hotel_name': 'McKittrick hotel'}
{'tools': {'messages': [ToolMessage(content="Error: ValueError('Unsupported response type: ')\n Please fix your mistakes.", name='book_hotel', id='f8b8c788-64c1-4cee-9f16-b4b7c78f63b3', tool_call_id='cf91d58a-c7ed-4d26-8906-